In [ ]:
# Import packages 
import sys
sys.path.append('../Plotting/')
""" Now you can imprt modules in ../Plotting"""

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.tri as tri
import matplotlib.colors as colors

from scipy.io import FortranFile

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import ESMF as E

import importlib
import glob
import copy

import xyp_plot as xyp
import scripGen as SG
import esmfRegrid as erg


importlib.reload( erg )
importlib.reload(SG)

In [ ]:
srcScrip = '/glade/work/juliob/SAMwrf_grids/SAMwrf01_ne30x16_np4_SCRIP.nc'
srcType = 'mesh'

In [ ]:
# ------- CAM SE ne30pg3 Scrip file
scripdir='/glade/p/cesmdata/cseg/inputdata/share/scripgrids/'
ne30scrip  = scripdir +  "ne30pg3_scrip_170611.nc"

dstScrip=ne30scrip
dstType='mesh'
ds_dstScrip = xr.open_dataset( ne30scrip )
area_dst = ds_dstScrip['grid_area'].values


In [ ]:
# Get default CAM Topo bottom BCs for this grid
bnd_topo = \
'/glade/p/cgd/amp/juliob/bndtopo/latest/ne30pg3_gmted2010_modis_bedmachine_nc3000_Laplace0100_20230105.nc'
dsTopo_CAM=xr.open_dataset( bnd_topo )
phis_CAM = dsTopo_CAM['PHIS'].values
lon_CAM = dsTopo_CAM['lon'].values
lat_CAM = dsTopo_CAM['lat'].values
area_CAM = dsTopo_CAM['area'].values


In [ ]:
wgts_file_BiL = '/glade/work/juliob/SAMwrf_grids/SAMwrf_ne30x16_X_ne30pg3-BiLinear_wgts.nc'
wgts_file_Con = '/glade/work/juliob/SAMwrf_grids/SAMwrf_ne30x16_X_ne30pg3-Conserve_wgts.nc'

In [ ]:
# Conservative regridding
regrd, srcf, dstf = erg.Regrid( srcScrip = srcScrip , 
                                srcType  = srcType  ,
                                dstScrip = dstScrip ,
                                dstType  = dstType  ,
                                RegridMethod='CONSERVE',
                                write_weights = False ,
                                read_weights = True ,
                                weights_file = wgts_file_Con )


In [ ]:
input_dir = \
    "/glade/campaign/cgd/projects/NCGD0051/ENSO_2010/L32/f.e22r.SAMwrf01.ne30x16.L32.NODEEP_2010_01/atm/hist/"
srcFile = input_dir+'f.e22r.SAMwrf01.ne30x16.L32.NODEEP_2010_01.cam.h0.2010-06.nc'

In [ ]:
ds_src   = xr.open_dataset(srcFile)
phis_src = ds_src['PHIS'].values
area_src = ds_src['area'].values

In [ ]:
np.shape(dstf.data)

In [ ]:
ncol_dst = np.shape(dstf.data)
ncol_src = np.shape(phis_src)
#try function
#########################
phis_src_x_dst = erg.HorzRG( aSrc = phis_src , 
                              regrd = regrd , 
                              srcField=srcf , 
                              dstField=dstf , 
                              srcShape=[ncol_src], 
                              dstShape=[ncol_dst] , 
                              srcGridkey='c', 
                              dstGridkey='c' )


In [ ]:
#plt.plot( phis_src_x_dst )

In [ ]:
plt.tricontourf(lon_CAM,lat_CAM,phis_src_x_dst)

In [ ]:
#

In [ ]:
print( np.sum( area_src * phis_src) )

In [ ]:
print( np.sum( area_CAM * phis_src_x_dst) )

In [ ]:
print( (np.sum( area_CAM * phis_src_x_dst) - np.sum( area_src * phis_src) ) /
     np.sum( area_CAM * phis_src_x_dst) )

"mass" error in Bi-Linear    = -0.0014088418151839306

"mass" error in Conservative = -6.589829802744967e-06
